# Clean Web Scrapped CFB Data

The purpose of this file is to clean CFB data that was previously webscarped to be used in further analysis latter on.
All data is coming from https://www.sports-reference.com

### Packages

***

In [1]:
# Importing Packages
import pandas as pd
import numpy as np 
import os
import time
from datetime import datetime

### Data Preperation

***

#### Directories

In [2]:
# File Directories 
base_path = os.getcwd()
save_folder = os.path.join(base_path, 'html_data')
file_name = 'webscrapped_cfb_stats_2022-2023.csv'
file_path = os.path.join(save_folder, file_name)

#### Reading In File

In [3]:
# Reading in file
stats = pd.read_csv(file_path)
stats.head(2)

,Rk_off,Date,Unnamed: 2_level_1_off,Result,Passing_Cmp_off,Passing_Att_off,Passing_Pct_off,Passing_Yds_off,Passing_TD_off,Rushing_Att_off,...,Penalties_No._def,Penalties_Yds_def,Fum,Int,TO,Season,is_post_season,Team,Conf,Opponent
0,1.0,2022-09-03,NaN,W (48-17),3,6,50.0,109,1,62,...,2,15,1,1,2,2022,False,Air Force,MWC (Mountain),Northern Iowa
1,2.0,2022-09-10,NaN,W (41-10),1,5,20.0,8,0,70,...,4,40,2,1,3,2022,False,Air Force,MWC (Mountain),Colorado


#### Updating column naming

In [4]:
# Renaming Columns to match snake case
stats.columns = [col.lower().replace(" ","_") for col in stats.columns]

# Updating Data Types
stats['date'] = pd.to_datetime(stats['date'])

stats.head(2)

,rk_off,date,unnamed:_2_level_1_off,result,passing_cmp_off,passing_att_off,passing_pct_off,passing_yds_off,passing_td_off,rushing_att_off,...,penalties_no._def,penalties_yds_def,fum,int,to,season,is_post_season,team,conf,opponent
0,1.0,2022-09-03,NaN,W (48-17),3,6,50.0,109,1,62,...,2,15,1,1,2,2022,False,Air Force,MWC (Mountain),Northern Iowa
1,2.0,2022-09-10,NaN,W (41-10),1,5,20.0,8,0,70,...,4,40,2,1,3,2022,False,Air Force,MWC (Mountain),Colorado


#### Infering Home or Away

In [5]:
# Adding Home and Away Indicator 
stats['location'] = np.select(
    [stats['unnamed:_2_level_1_off'] == '@'
     ,stats['unnamed:_2_level_1_off'] == 'N']
    ,['Away', 'Netural']
    ,default = 'Home'
)

# Removing Unnamed and Index columns
columns_to_drop = [l for l in stats.columns if 'unnamed' in l or 'rk' in l]
stats = stats.drop(columns=columns_to_drop, axis=1)
stats.head(2)

,date,result,passing_cmp_off,passing_att_off,passing_pct_off,passing_yds_off,passing_td_off,rushing_att_off,rushing_yds_off,rushing_avg_off,...,penalties_yds_def,fum,int,to,season,is_post_season,team,conf,opponent,location
0,2022-09-03,W (48-17),3,6,50.0,109,1,62,582,9.4,...,15,1,1,2,2022,False,Air Force,MWC (Mountain),Northern Iowa,Home
1,2022-09-10,W (41-10),1,5,20.0,8,0,70,435,6.2,...,40,2,1,3,2022,False,Air Force,MWC (Mountain),Colorado,Home


#### Renaming 0ffensive and Defensive Columns

In [6]:
# Remaping Columns 
column_map = {
      'date': 'date'
     ,'result': 'result'
     ,'passing_cmp_off': 'passing_completion'
     ,'passing_att_off': 'passing_attempts'
     ,'passing_yds_off': 'yards_gained_passing'
     ,'passing_td_off': 'tocuhdowns_passing'
     ,'rushing_att_off': 'rushing_attempts'
     ,'rushing_yds_off': 'yards_gained_rushing'
     ,'rushing_td_off': 'tocuhdowns_rushing'
     ,'total_offense_plays_off': 'total_offensive_plays'
     ,'total_offense_yds_off': 'total_offensive_yards'
     ,'first_downs_pass_off': 'first_down_gained_by_passing'
     ,'first_downs_rush_off': 'first_down_gained_by_rushing'
     ,'first_downs_pen_off': 'first_down_gained_by_penality'
     ,'first_downs_tot_off': 'total_first_downs_gained'
     ,'penalties_no._off': 'number_of_offensive_penality'
     ,'penalties_yds_off': 'offensive_penality_yards'
     ,'turnovers_fum': 'fumbles'
     ,'turnovers_int': 'interceptions'
     ,'turnovers_tot': 'turnovers'
     ,'passing_cmp_def': 'passes_completed_against'
     ,'passing_att_def': 'passes_attempted_against'
     ,'passing_yds_def': 'yards_allowed_passing'
     ,'passing_td_def': 'touch_downs_allowed_passing'
     ,'rushing_att_def': 'rushes_attempted_against'
     ,'rushing_yds_def': 'yards_allowed_rushing'
     ,'rushing_td_def': 'touch_downs_allowed_rushing'
     ,'total_offense_plays_def': 'total_defense_plays'
     ,'total_offense_yds_def': 'total_yards_allowed'
     ,'first_downs_pass_def': 'first_downs_allowed_by_pass'
     ,'first_downs_rush_def': 'first_downs_allowed_by_rush'
     ,'first_downs_pen_def': 'first_downs_allowed_by_penalty'
     ,'first_downs_tot_def': 'total_first_downs_allowed'
     ,'penalties_no._def': 'number_of_defense_penality'
     ,'penalties_yds_def': 'defensse_penality_yards'
     ,'fum': 'fumbles_forced'
     ,'int': 'interception_forced'
     ,'to': 'total_turnovers_forced'
     ,'season': 'season'
     ,'is_post_season': 'is_post_season'
     ,'team': 'team'
     ,'conf': 'conf'
     ,'opponent': 'opponent'
     ,'location': 'location'
}
stats = stats.rename(
     columns = column_map
    )

stats.head(2)

,date,result,passing_completion,passing_attempts,passing_pct_off,yards_gained_passing,tocuhdowns_passing,rushing_attempts,yards_gained_rushing,rushing_avg_off,...,defensse_penality_yards,fumbles_forced,interception_forced,total_turnovers_forced,season,is_post_season,team,conf,opponent,location
0,2022-09-03,W (48-17),3,6,50.0,109,1,62,582,9.4,...,15,1,1,2,2022,False,Air Force,MWC (Mountain),Northern Iowa,Home
1,2022-09-10,W (41-10),1,5,20.0,8,0,70,435,6.2,...,40,2,1,3,2022,False,Air Force,MWC (Mountain),Colorado,Home


#### Determing Winner of Game

In [7]:
# Spliting Results Column and Determinig Winner
# Spliting Score
chars_to_remove = "W|L|[()]"
stats['result'] = stats['result'].str.replace(chars_to_remove,"", regex=True)
stats['result'] = stats['result'].str.strip()
stats[['points_for', 'points_against']] = stats['result'].str.split("-", expand=True)

# Fixing Types
stats['points_for'] = stats['points_for'].astype(int)
stats['points_against'] = stats['points_against'].astype(int)

# Determing Winner
stats['is_win'] = stats['points_for'] > stats['points_against']
stats = stats.drop(columns="result", axis=1)

stats.head(2)

,date,passing_completion,passing_attempts,passing_pct_off,yards_gained_passing,tocuhdowns_passing,rushing_attempts,yards_gained_rushing,rushing_avg_off,tocuhdowns_rushing,...,total_turnovers_forced,season,is_post_season,team,conf,opponent,location,points_for,points_against,is_win
0,2022-09-03,3,6,50.0,109,1,62,582,9.4,5,...,2,2022,False,Air Force,MWC (Mountain),Northern Iowa,Home,48,17,True
1,2022-09-10,1,5,20.0,8,0,70,435,6.2,5,...,3,2022,False,Air Force,MWC (Mountain),Colorado,Home,41,10,True


#### Reordering Data

In [8]:
# Reording Data 
new_order = [
     'date'
    ,'conf'
    ,'season'
    ,'team'
    ,'opponent'
    ,'location'
    ,'points_for'
    ,'points_against'
    ,'is_win'
    ,'is_post_season'
    ,'passing_completion'
    ,'passing_attempts'
    ,'yards_gained_passing'
    ,'tocuhdowns_passing'
    ,'rushing_attempts'
    ,'yards_gained_rushing'
    ,'tocuhdowns_rushing'
    ,'total_offensive_plays'
    ,'total_offensive_yards'
    ,'first_down_gained_by_passing'
    ,'first_down_gained_by_rushing'
    ,'first_down_gained_by_penality'
    ,'total_first_downs_gained'
    ,'number_of_offensive_penality'
    ,'offensive_penality_yards'
    ,'fumbles'
    ,'interceptions'
    ,'turnovers'
    ,'passes_completed_against'
    ,'passes_attempted_against'
    ,'yards_allowed_passing'
    ,'touch_downs_allowed_passing'
    ,'rushes_attempted_against'
    ,'yards_allowed_rushing'
    ,'touch_downs_allowed_rushing'
    ,'total_defense_plays'
    ,'total_yards_allowed'
    ,'first_downs_allowed_by_pass'
    ,'first_downs_allowed_by_rush'
    ,'first_downs_allowed_by_penalty'
    ,'total_first_downs_allowed'
    ,'number_of_defense_penality'
    ,'defensse_penality_yards'
    ,'fumbles_forced'
    ,'interception_forced'
    ,'total_turnovers_forced'
]
stats = stats.reindex(columns=new_order)
stats.head(2)

,date,conf,season,team,opponent,location,points_for,points_against,is_win,is_post_season,...,total_yards_allowed,first_downs_allowed_by_pass,first_downs_allowed_by_rush,first_downs_allowed_by_penalty,total_first_downs_allowed,number_of_defense_penality,defensse_penality_yards,fumbles_forced,interception_forced,total_turnovers_forced
0,2022-09-03,MWC (Mountain),2022,Air Force,Northern Iowa,Home,48,17,True,False,...,405,17,4,0,21,2,15,1,1,2
1,2022-09-10,MWC (Mountain),2022,Air Force,Colorado,Home,41,10,True,False,...,162,2,5,1,8,4,40,2,1,3


#### Exploring Missing Team Data

In [9]:
# Find Missing Defensive Data 
offensive_teams = stats[['team']].copy()
offensive_teams['Column'] = 'Team'
defensive_teams = stats[['opponent']].copy()
defensive_teams.columns = ['team']
defensive_teams['Column'] = 'Oppenent'
test = [offensive_teams, defensive_teams]
test = pd.concat(test)
test['count'] = 1
test = test.drop_duplicates().reset_index(drop=True)
pivoted = pd.pivot_table(
     test
    ,columns='Column'
    ,index='team'
    ,values='count'
    ,aggfunc='sum'
    ,fill_value=0
)
pivoted['Total'] = pivoted['Team'] + pivoted['Oppenent']
teams_missing_data = pivoted[pivoted['Total'] == 1]
teams_missing_data.head()

Column,Oppenent,Team,Total
team,,,
Abilene Christian,1,0,1
Alabama A&M,1,0,1
Alabama State,1,0,1
Alabama-Birmingham,1,0,1
Albany,1,0,1


### Data Sumarization

***

In [10]:
data_sumamry = stats.describe(include='all')
data_sumamry.loc['data_type'] = data_sumamry.dtypes
data_sumamry.loc['rows'] = len(stats)
data_sumamry.loc['percent_null'] = stats.isnull().mean()
data_sumamry

,date,conf,season,team,opponent,location,points_for,points_against,is_win,is_post_season,...,total_yards_allowed,first_downs_allowed_by_pass,first_downs_allowed_by_rush,first_downs_allowed_by_penalty,total_first_downs_allowed,number_of_defense_penality,defensse_penality_yards,fumbles_forced,interception_forced,total_turnovers_forced
count,3374,3374,3374.0,3374,3374,3374,3374.0,3374.0,3374,3374,...,3374.0,3374.0,3374.0,3374.0,3374.0,3374.0,3374.0,3374.0,3374.0,3374.0
unique,NaN,19,NaN,133,236,3,NaN,NaN,2,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
top,NaN,Pac-12,NaN,Georgia,Michigan,Home,NaN,NaN,True,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
freq,NaN,308,NaN,29,29,1685,NaN,NaN,1794,3206,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
mean,2023-04-19 23:36:31.582691328,NaN,2022.504446,NaN,NaN,NaN,28.082098,25.970362,NaN,NaN,...,375.096621,9.906935,8.186129,1.745702,19.838767,5.94754,51.82128,0.601956,0.838471,1.440427
min,2022-08-27 00:00:00,NaN,2022.0,NaN,NaN,NaN,0.0,0.0,NaN,NaN,...,50.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0
25%,2022-10-15 00:00:00,NaN,2022.0,NaN,NaN,NaN,17.0,16.0,NaN,NaN,...,295.0,7.0,5.0,1.0,16.0,4.0,33.0,0.0,0.0,0.0
50%,2023-08-31 00:00:00,NaN,2023.0,NaN,NaN,NaN,27.0,24.5,NaN,NaN,...,372.0,10.0,8.0,1.0,20.0,6.0,50.0,0.0,1.0,1.0
75%,2023-10-14 00:00:00,NaN,2023.0,NaN,NaN,NaN,38.0,35.0,NaN,NaN,...,453.0,13.0,11.0,3.0,24.0,8.0,68.0,1.0,1.0,2.0
max,2024-01-08 00:00:00,NaN,2023.0,NaN,NaN,NaN,81.0,77.0,NaN,NaN,...,763.0,27.0,28.0,10.0,39.0,18.0,216.0,5.0,5.0,8.0


### Exporting the Data

***

In [11]:
used_stats_save_path = os.path.join(save_folder, 'cleaned_cfb_data.csv')
stats.to_csv(used_stats_save_path, index=False)